In [4]:
import numpy as np
import pandas as pd
import pyodbc
from Username_PW import *

Ran my query to get the data I need for doorswings at one particular store

In [11]:
link = 'DRIVER={DRIVERNAME};DBCNAME={hostname};Authentication=LDAP; UID={uid};PWD={pwd}'.format(
    DRIVERNAME='Teradata Database ODBC Driver 16.20', hostname='172.28.130.20',
    uid= username ,Database='WAVE_DAILY_XFINITY_STORES', pwd= password)

conn = pyodbc.connect(link)

cus=conn.cursor()
print('Connection to Teradata NDW established')

sql_query = pd.read_sql_query("""

SELECT SL.LOCATION_DESC
,ST.STORE_TRAFFIC_DATE
--,ST.STORE_TRAFFIC_TIME
,SUM(ST.EXIT_CNT) as Doorswings
FROM	NDW_SALESCOMP_ANALYTICS_VIEWS.WAVE_DAILY_XFINITY_STORE_TRAFFIC as ST
INNER JOIN NDW_ARTERRA_JRNL_VIEWS.WAVE_DAILY_XFINITY_STORE_LISTING as SL
ON ST.COMCAST_STORE_ID = SL.COMCAST_STORE_ID
WHERE TRAFFIC_TYPE_CODE = 'Actual'
AND ST.COMCAST_STORE_ID = '3351'

ORDER BY
ST.COMCAST_STORE_ID
,SL.LOCATION_DESC
,ST.STORE_TRAFFIC_DATE
--,ST.STORE_TRAFFIC_TIME
,Doorswings

GROUP BY
ST.COMCAST_STORE_ID
,SL.LOCATION_DESC
,ST.STORE_TRAFFIC_DATE
--,ST.STORE_TRAFFIC_TIME
""", conn)

#cus.execute(query_ar)

#row = cus.fetchone()
#print(row)

df = pd.DataFrame(sql_query, columns = ['LOCATION_DESC', 'STORE_TRAFFIC_DATE','Doorswings'])

conn.close()
print("Connection Closed")

print(df)

Connection to Teradata NDW established
Connection Closed
                    LOCATION_DESC STORE_TRAFFIC_DATE  Doorswings
0     3351 - Albuquerque, NM (XF)         2014-12-22         656
1     3351 - Albuquerque, NM (XF)         2014-12-23         612
2     3351 - Albuquerque, NM (XF)         2014-12-24         354
3     3351 - Albuquerque, NM (XF)         2014-12-25           0
4     3351 - Albuquerque, NM (XF)         2014-12-26         819
...                           ...                ...         ...
2817  3351 - Albuquerque, NM (XF)         2022-10-28         539
2818  3351 - Albuquerque, NM (XF)         2022-10-29         427
2819  3351 - Albuquerque, NM (XF)         2022-10-30         269
2820  3351 - Albuquerque, NM (XF)         2022-10-31         472
2821  3351 - Albuquerque, NM (XF)         2022-11-01           0

[2822 rows x 3 columns]


I wanted to see if there were any null values as well as the data types.

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2822 entries, 0 to 2821
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   LOCATION_DESC       2822 non-null   object
 1   STORE_TRAFFIC_DATE  2822 non-null   object
 2   Doorswings          2822 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 66.3+ KB


I renamed the columns so that it would be easier to type out

In [20]:
df2 = df.rename(columns={'LOCATION_DESC':'store_name', 'STORE_TRAFFIC_DATE':'date', 'Doorswings':'doorswings'})

In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2822 entries, 0 to 2821
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   store_name  2822 non-null   object
 1   date        2822 non-null   object
 2   doorswings  2822 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 66.3+ KB


In [22]:
df2.isnull().sum()

store_name    0
date          0
doorswings    0
dtype: int64

I ran describe to understand at a high level what the statistics were behind doorswings

In [23]:
df2.doorswings.describe()

count    2822.000000
mean      609.957831
std       203.417848
min         0.000000
25%       522.500000
50%       619.500000
75%       730.000000
max      1291.000000
Name: doorswings, dtype: float64

I wanted to see what values were recurring in the dataset

In [24]:
df2.doorswings.value_counts()

0      28
550    14
581    14
619    13
599    13
       ..
386     1
917     1
904     1
501     1
441     1
Name: doorswings, Length: 793, dtype: int64